In [150]:
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from csv import writer
from sklearn.feature_extraction.text import TfidfTransformer

finwiz_url = 'https://finviz.com/quote.ashx?t='


In [149]:
news_tables = {}
tickers = ['AMZN', 'TSLA', 'GOOG']

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url, headers={'user-agent': 'my-app/0.0.1'})
    response = urlopen(req)
    # Read the contents of the file into 'html'
    html = BeautifulSoup(response)
    # Find 'news-table' in the Soup and load it into 'news_table'
    news_table = html.find(id='news-table')
    # Add the table to our dictionary
    news_tables[ticker] = news_table

In [151]:
parsed_news = []

# Iterate through the news
for file_name, news_table in news_tables.items():
    # Iterate through all tr tags in 'news_table'
    for x in news_table.findAll('tr'):
        # read the text from each tr tag into text
        # get text from a only
        text = x.a.get_text()
        # splite text in the td tag into a list
        date_scrape = x.td.text.split()
        # if the length of 'date_scrape' is 1, load 'time' as the only element

        if len(date_scrape) == 1:
            time = date_scrape[0]

        # else load 'date' as the 1st element and 'time' as the second
        else:
            date = date_scrape[0]
            time = date_scrape[1]
        # Extract the ticker from the file name, get the string up to the 1st '_'
        ticker = file_name.split('_')[0]

        # Append ticker, date, time and headline as a list to the 'parsed_news' list
        parsed_news.append([ticker, date, time, text])

In [152]:
columns = ['ticker', 'date', 'time', 'headline']

# Convert the parsed_news list into a DataFrame called 'parsed_and_scored_news'
parsed_and_scored_news = pd.DataFrame(parsed_news, columns=columns)

# Web scraping taken from TowardsDataScience from Damien Boh
'Damian Boh,May 6 (2020),Sentiment Analysis of Stocks from Financial News using Python,[Source code],https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638'

'Damian Boh,May 6 (2020),Sentiment Analysis of Stocks from Financial News using Python,[Source code],https://towardsdatascience.com/sentiment-analysis-of-stocks-from-financial-news-using-python-82ebdcefb638'

In [153]:
parsed_and_scored_news 

,ticker,date,time,headline
0,AMZN,Jul-14-20,02:49PM,"US STOCKS-Dow jumps 1.6%, leads Wall Street hi..."
1,AMZN,Jul-14-20,02:05PM,Walmart Leads $1.2 Billion Funding Round of Fl...
2,AMZN,Jul-14-20,02:02PM,Dow Jones Jumps as JPMorgan Beats Earnings Est...
3,AMZN,Jul-14-20,01:19PM,Amazon Workers in Memphis Are Spooked as Virus...
4,AMZN,Jul-14-20,12:20PM,Amazon Applies 'Just Walk Out' Technology to S...
...,...,...,...,...
295,GOOG,Jul-09-20,10:59AM,Exclusive: Google can ward off EU antitrust pr...
296,GOOG,Jul-09-20,10:59AM,Exclusive: Google can ward off EU antitrust pr...
297,GOOG,Jul-09-20,10:36AM,Round two of stimulus is 'already priced into ...
298,GOOG,Jul-09-20,09:53AM,Google Cloud Forges Multi-Year Deal With Renault


In [226]:
parsed_and_scored_news


data_test = parsed_and_scored_news[parsed_and_scored_news['date'] > 'Jul-13-20'].copy()


headlines = data_test['headline']

In [227]:
df = pd.read_csv('stock_datafinal.csv')


In [228]:
positive = df[df['Sentiment']==1]
Negative = df[df['Sentiment']==0]
positive.shape

(3341, 2)

In [229]:
Negative.shape

(1634, 2)

In [230]:
positive = positive.sample(Negative.shape[0])

In [231]:
positive.shape

(1634, 2)

In [232]:
data = positive.append(Negative,ignore_index=True)
data.tail()

,Text,Sentiment
3263,"JOSB 50% off on almost everything. Funny, they...",0
3264,DE considering buying some put options...,0
3265,Happy Easter! SHOT Setups: CS ISG CF AN NKD OW...,0
3266,CAT may try to jump on the boat with some Put ...,0
3267,"If AAP goes to over 451, I will go short. Stil...",0


In [233]:
X_train, X_test, y_train, y_test = train_test_split(data['Text'], data['Sentiment'], test_size=.1, random_state=0,
                                                    shuffle=True,stratify=data['Sentiment'] )

In [234]:
clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', SVC(C=10000, gamma='auto'))])

In [235]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=10000, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_fu

In [236]:
y_pred = clf.predict(X_test)


In [237]:
confusion_matrix(y_test, y_pred)


array([[126,  38],
       [ 41, 122]])

In [238]:
accuracy_score(y_test, y_pred)

0.7584097859327217

In [300]:
pos = []
neg = []
error = []
ticker2 = []
realticker =[]

In [301]:
data_test.shape[0]

91

In [302]:
for i in range(0,data_test.shape[0]):
  try:
      if clf.predict([data_test['headline'][i]])==1:
        ticker2=[data_test['ticker'][i]]
        realticker.append(ticker2)
        pos.append('1')
      else: 
        neg.append('0')
  except KeyError:
    error.append('error')
 
  

In [303]:
len(pos)

23

In [304]:
len(neg)

12

In [305]:
len(error)

56

In [306]:
realticker

[['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN'],
 ['AMZN']]